In [111]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense , Reshape , LeakyReLU , BatchNormalization , Conv2DTranspose ,Conv2D , Flatten , Input
from keras.models import Sequential , Model
from keras.losses import binary_crossentropy
from keras.datasets import mnist
from keras.utils import plot_model
import warnings

In [112]:
warnings.filterwarnings("ignore")

## Data Loading

In [113]:
(X_train , y_train ),(X_test , y_test) = mnist.load_data();

In [114]:
X_train.shape

(60000, 28, 28)

In [115]:
X_train = (X_train -127.5) / 127.5
print(X_train.max() ,'\n', X_train.min())

1.0 
 -1.0


In [116]:
total_epoch = 30
Batch_size = 256
Half_batch = 128
no_batchs = int(X_train.shape[0]/Batch_size)
NOISE_dim = 100

In [117]:
X_train.shape

(60000, 28, 28)

## Generator Model

In [118]:
generator = Sequential()
generator.add(Dense(units = 7*7*128 , input_shape=(NOISE_dim,)))
generator.add(Reshape((7,7,128)))
generator.add(LeakyReLU(0.2))
generator.add(BatchNormalization())
generator.add(Conv2DTranspose(64,(3,3),strides=(2,2),padding="same"))
generator.add(LeakyReLU(0.2))
generator.add(BatchNormalization())
generator.add(Conv2DTranspose(1,(3,3),strides=(2,2),padding="same",activation="tanh"))

In [119]:
generator.compile(loss=binary_crossentropy,optimizer=keras.optimizers.Adam(learning_rate=2e-4,beta_1=0.5))

In [120]:
generator.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 6272)              633472    
                                                                 
 reshape_5 (Reshape)         (None, 7, 7, 128)         0         
                                                                 
 leaky_re_lu_25 (LeakyReLU)  (None, 7, 7, 128)         0         
                                                                 
 batch_normalization_10 (Ba  (None, 7, 7, 128)         512       
 tchNormalization)                                               
                                                                 
 conv2d_transpose_10 (Conv2  (None, 14, 14, 64)        73792     
 DTranspose)                                                     
                                                                 
 leaky_re_lu_26 (LeakyReLU)  (None, 14, 14, 64)      

##  Discriminator Model

In [121]:
discriminator = Sequential()
discriminator.add(Conv2D(64,kernel_size=(3,3),strides=(2,2),padding="same",input_shape=(28,28,1)))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Conv2D(128,kernel_size=(3,3),strides=(2,2),padding="same"))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Flatten())
discriminator.add(Dense(100))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dense(1,activation='sigmoid'))

In [122]:
discriminator.compile(loss=binary_crossentropy,optimizer=keras.optimizers.Adam(learning_rate=2e-4,beta_1=0.5))

In [123]:
discriminator.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 14, 14, 64)        640       
                                                                 
 leaky_re_lu_27 (LeakyReLU)  (None, 14, 14, 64)        0         
                                                                 
 conv2d_11 (Conv2D)          (None, 7, 7, 128)         73856     
                                                                 
 leaky_re_lu_28 (LeakyReLU)  (None, 7, 7, 128)         0         
                                                                 
 flatten_5 (Flatten)         (None, 6272)              0         
                                                                 
 dense_16 (Dense)            (None, 100)               627300    
                                                                 
 leaky_re_lu_29 (LeakyReLU)  (None, 100)             

## Combine Models

In [124]:
discriminator.trainable = False

In [125]:
gan_input = Input(shape=(NOISE_dim,))

generator_img = generator(gan_input)

gan_out = discriminator(generator_img)

model = Model(gan_input , gan_out)

In [126]:
model.compile(loss=binary_crossentropy ,optimizer=keras.optimizers.Adam(learning_rate=2e-4,beta_1=0.5))

In [127]:
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential_10 (Sequential)  (None, 28, 28, 1)         708609    
                                                                 
 sequential_11 (Sequential)  (None, 1)                 701897    
                                                                 
Total params: 1410506 (5.38 MB)
Trainable params: 708225 (2.70 MB)
Non-trainable params: 702281 (2.68 MB)
_________________________________________________________________


## Training

In [128]:
X_train = X_train.reshape(-1,28,28,1)

In [132]:
d_losses = [ ]
g_losses = [ ]
for epoch in range(total_epoch):
  epoch_d_loss = 0.0
  epoch_g_loss = 0.0

  # Mini batch
  for step in range(no_batchs):
    # real images
    discriminator.trainable = True
    idx = np.random.randint(0,60000,Half_batch)
    true_img = X_train[idx]
    # fake images
    noise = np.random.normal(0,1,size=(Half_batch,NOISE_dim))
    fake_img = generator.predict(noise)
    # labels
    true_y = np.ones((Half_batch,1)) * 0.9
    fake_y = np.zeros((Half_batch,1))
    # Train descriminator
    d_loss_real = discriminator.train_on_batch(true_img ,true_y)
    d_loss_fake = discriminator.train_on_batch(fake_img ,fake_y)

    d_loss = 0.5 * d_loss_real + 0.5 * d_loss_fake
    epoch_d_loss += d_loss

    # Train generator
    discriminator.trainable = False

    noise = np.random.normal(0,1,size=(Batch_size, NOISE_dim))
    ground_truth_y = np.ones((Batch_size,1))
    g_loss = model.train_on_batch(noise , ground_truth_y)
    epoch_g_loss += g_loss


    print(f'Epoch{epoch+1} , Descriminator loss {epoch_d_loss  / no_batchs }, Generator loss {epoch_g_loss / no_batchs }')

    d_losses.append(epoch_d_loss  / no_batchs)
    g_losses.append(epoch_g_loss /no_batchs)

    if (epoch+1) % 10 == 0:
      generator.save("generator.h5")


Streaming output truncated to the last 5000 lines.
4/4 [==============================] - 0s 3ms/step
Epoch20 , Descriminator loss 0.19285141135382855, Generator loss 0.41700720430439353
4/4 [==============================] - 0s 3ms/step
Epoch20 , Descriminator loss 0.19536510517454556, Generator loss 0.4227183659871419
4/4 [==============================] - 0s 5ms/step
Epoch20 , Descriminator loss 0.1979320559364099, Generator loss 0.42830611599816215
4/4 [==============================] - 0s 3ms/step
Epoch20 , Descriminator loss 0.20052573753473085, Generator loss 0.434052369533441
4/4 [==============================] - 0s 4ms/step
Epoch20 , Descriminator loss 0.2031461267899244, Generator loss 0.43983293394757134
4/4 [==============================] - 0s 3ms/step
Epoch20 , Descriminator loss 0.20569172616188341, Generator loss 0.4452376574532598
4/4 [==============================] - 0s 4ms/step
Epoch20 , Descriminator loss 0.20827473012300637, Generator loss 0.45064686404334175
4/4